<div style="background-color:#000;"><img src="pqn.png"></img></div><div><a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.</div>

## Library installation

Install the libraries we need to download market data, fit a GARCH volatility model, and work with arrays and plots in a notebook.

In [ ]:
!pip install numpy yfinance arch matplotlib

The arch package includes compiled components on some platforms, so if pip fails, try upgrading pip/wheel or installing via conda-forge in a conda environment.

## Imports and setup

We use numpy for numeric transforms, yfinance to pull “AAPL” price history, arch to fit a GARCH model, and matplotlib to set plot styling for any visuals we add later.

In [ ]:
import numpy as np
import yfinance as yf
from arch import arch_model

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rc("figure", figsize=(16, 6))
plt.rc("font", family="sans-serif")
plt.rc("font", size=14)

Setting plot defaults early keeps our workflow consistent when we later compare volatility estimates or option values visually. In research notebooks, clean, repeatable plotting reduces the chance we misread regime changes because of inconsistent chart scales. It also helps when we start validating whether a volatility input is “too slow” after a market shift.

## Download prices and compute returns

Pull adjusted close prices with yfinance and convert them into daily percentage returns, which is the standard input format for GARCH-style volatility models.

In [ ]:
prices = yf.download("AAPL", start="2020-01-01", end="2024-12-31", auto_adjust=True).Close
returns = 100 * prices.pct_change().dropna()

Using adjusted close helps us avoid artificial jumps from splits and dividends, which can contaminate volatility estimation. We scale returns by 100 because many volatility libraries (including arch) conventionally work in percent units, and it makes model outputs easier to sanity-check. This step is where beginners often introduce quiet-regime bias by choosing too short or too cherry-picked a window, so it’s worth being deliberate about dates.

## Fit GARCH to forecast volatility

Fit a default GARCH model with arch and generate a one-step-ahead conditional variance forecast for the next trading day.

In [ ]:
model = arch_model(returns)
res = model.fit()
forecast = res.forecast(horizon=1, reindex=False)
variance_forecast = forecast.variance.iat[-1, 0]

This is the core shift from “volatility as a single historical statistic” to “volatility as a forecast conditioned on recent shocks.” In options work, the difference matters because pricing is convex in volatility, so stale inputs can systematically misprice premiums when regimes change. Default settings are fine for a first pass, but in practice we validate residuals and experiment with distributions (e.g., Student-t) before trusting the forecast in production.

## Convert variance into annual volatility

Convert the variance forecast into a daily volatility estimate, then annualize it to match how options quotes and vol surfaces are typically expressed.

In [ ]:
volatility_forecast = np.sqrt(variance_forecast)
annualized_volatility_forecast = volatility_forecast * np.sqrt(252) / 100

Taking the square root moves us from variance to volatility, which is what traders usually think in and what plugs into models like Black-Scholes. Annualizing with 252 trading days gives a like-for-like number to compare against implied volatility or a historical annualized standard deviation. This is also where unit consistency bites beginners: because we modeled percent returns, we divide by 100 when turning the annualized figure into a decimal volatility for pricing formulas.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://www.pyquantnews.com/getting-started-with-python-for-quant-finance/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.